In [ ]:
# Imports
# Standard Libraries of Python
import os
import sys
import time
import itertools
from collections import Counter
from decimal import Decimal, ROUND_HALF_UP, getcontext
getcontext().prec = 5

# Dependencies
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series

# Libraries made for this Proyect
from src.parse import draw_generator,Criteria
ruta_carpeta = 'data/simulation_result/'

In [ ]:
# Test of Tomorrow Numbers
euromillions = Criteria()

euromillions.groups_info()
euromillions.apply_transformation()
euromillions.count_skips()
euromillions.skips_for_last_12_draws()
euromillions.get_natural_rotations()
euromillions.numbers_clasification()

euromillions.year_criterion()
euromillions.rotation_criterion()
euromillions.position_criterion()
euromillions.group_criterion()
euromillions.numbers_of_tomorrow()

In [ ]:
# Classes to be defined
class Tickets():
    def __init__(self,euromillions):
        self.recommended_numbers = euromillions.recommended_numbers
        self.not_recommended_numbers = euromillions.not_recommended_numbers
        self.counts = euromillions.counts
        self.skips_7_12 = euromillions.skips_7_12
        self._selected_numbers = []

    def draw_skips(self) -> DataFrame:
        self.d_skips = pd.DataFrame(columns=['nro1','nro2','nro3','nro4','nro5'])
        for index, row in self.counts.iterrows():
            new_row = []

            for column, value in row.items():
                if value == 0:
                    try:
                        aus = self.counts.loc[index - 1, column]
                    except KeyError:
                        aus = 0
                    if pd.isna(aus):
                        aus = 0
                    new_row.append(aus)
        
            while len(new_row) < 5:
                new_row.append(0)
        
            self.d_skips.loc[index] = new_row
        
    def skips_evaluation(self) -> DataFrame:
        self.evaluation = pd.DataFrame(columns=['0','5','7','10','13'])
        counts = pd.DataFrame(0, index=self.d_skips.index, columns=self.evaluation.columns)

        counts['0'] = self.d_skips.apply(lambda row: row.eq(0).sum(),axis=1)
        counts['5'] = self.d_skips.apply(lambda row: row.between(0,5).sum(),axis=1)
        counts['7'] = self.d_skips.apply(lambda row: row.between(0,7).sum(),axis=1)
        counts['10'] = self.d_skips.apply(lambda row: row.between(0,10).sum(),axis=1)
        counts['13'] = self.d_skips.apply(lambda row: row.between(0,13).sum(),axis=1)

        self.evaluation = pd.concat([self.evaluation,counts],ignore_index=True)
        self.evaluation = self.evaluation.set_index(pd.RangeIndex(1, len(self.evaluation) + 1))

    def __df_numbers(self) -> Series:
        self._df_values = euromillions.last_draw.groupby('skips')['number'].apply(lambda x: list(x)).reset_index().set_index('skips')
        self._df_values = self._df_values.rename_axis('number').rename_axis(None)
        self._df_values['number'] = self._df_values['number'].apply(lambda x: sorted(x))
        return self._df_values['number']
    
    def __remove_number(self, df:DataFrame, number:int) -> DataFrame:
        if df.equals(self.recommended_numbers):
            self.recommended_numbers = df.drop(df[df['numbers'] == number].index).reset_index(drop=True)
            probability = 1 / len(df)
            self.recommended_numbers['criteria'] = self.recommended_numbers['criteria'] * (1 + probability)
            return self.recommended_numbers
        else:
            self.not_recommended_numbers = df.drop(df[df['numbers'] == number].index).reset_index(drop=True)
            probability = 1 / len(df)
            self.not_recommended_numbers['criteria'] = self.not_recommended_numbers['criteria'] * (1 + probability)
            return self.not_recommended_numbers
    
    def __select_number(self, idx:int, numbers:DataFrame) -> int:
        l = sorted(euromillions.last_draw[euromillions.last_draw['skips'] == idx]['number'].to_list())
        available_numbers = numbers[numbers['numbers'].isin(l)]['numbers'].tolist()
        if available_numbers:
            n = np.random.choice(available_numbers)
            self._selected_numbers.append(n)
            self.__remove_number(numbers,n)

    def first_number(self) -> int:
        self.numbers = self.__df_numbers()
        available_numbers = self.numbers.loc[0]
        if available_numbers:
            self.__select_number(0,self.recommended_numbers)
        else:
            self.__select_number(0,self.not_recommended_numbers)
    
    def suggested_numbers(self):
        recommended_numbers_selected = 0
        while recommended_numbers_selected < 6:
            skip_0 = self.skips_7_12[self.skips_7_12['7'] == 0].index.to_list()
            idx = np.random.choice(skip_0)
            if idx > 12 and len(euromillions.last_draw[euromillions.last_draw['skips'] < 12]) < 39:
                self.__select_number(idx,self.recommended_numbers)
            else:
                idx = np.random.choice(range(13),size=1)
                self.__select_number(idx,self.recommended_numbers)
                

In [ ]:
lotto = Tickets(euromillions)

In [ ]:
lotto.draw_skips()
lotto.d_skips

In [ ]:
lotto.skips_evaluation()
lotto.evaluation

In [ ]:
lotto.first_number()

In [ ]:
lotto._selected_numbers

In [ ]:
lotto.recommended_numbers

In [ ]:
df_values = euromillions.last_draw.groupby('skips')['number'].apply(lambda x: list(x)).reset_index().set_index('skips')
df_values = df_values.rename_axis('number').rename_axis(None)
df_values['number'] = df_values['number'].apply(lambda x: sorted(x))

In [ ]:
df_values

In [ ]:
selected_numbers = []
zero_rows_selected = 0
numbers = df_values['number']
for idx, column_7 in euromillions.skips_7_12['7'].items():

    match idx:
        case 0:
            select_last_draw = numbers.loc[idx]
            available_numbers = [number for number in select_last_draw if number in euromillions.recommended_numbers['numbers'].values]
            if available_numbers:
                selected_number = np.random.choice(available_numbers)
                selected_numbers.append(selected_number)
            else:
                available_numbers = [number for number in select_last_draw if number in euromillions.not_recommended_numbers['numbers'].values]
                selected_number = np.random.choice(available_numbers)
                selected_numbers.append(selected_number)


In [ ]:
euromillions.recommended_numbers

In [ ]:
patata = [number for number in select_last_draw if number in euromillions.recommended_numbers['numbers'].values]

In [ ]:
type(patata)

In [ ]:
euromillions.not_recommended_numbers.drop(euromillions.not_recommended_numbers.iloc[0,0])

In [ ]:
df_values.loc[df_values.index <= 12,'number'].apply(len).sum() - df_values.loc[df_values.index == 12].apply(len)

In [ ]:
# Test of lottery tickets (dev)
lotto = Tickets(euromillions)
lotto.draw_skips()

In [ ]:
# Test of Euromillions Analysis
start_time = time.time()

success = []
failure = []
results = []
euromillions = Criteria()
size = len(euromillions.scrap)

for draw in draw_generator(size):
    db_slice = euromillions.scrap.head(draw)
    euromillions.db = db_slice
    euromillions.groups_info()
    euromillions.apply_transformation()
    euromillions.count_skips()
    euromillions.skips_for_last_12_draws()
    euromillions.get_natural_rotations()
    euromillions.numbers_clasification()

    euromillions.year_criterion()
    euromillions.rotation_criterion()
    euromillions.position_criterion()
    euromillions.group_criterion()
    euromillions.numbers_of_tomorrow()

    row = euromillions.scrap.loc[draw,['nro1','nro2','nro3','nro4','nro5']]
    column = euromillions.recommended_numbers.loc[:,'numbers']
    if len(column) < 25:
        missing_rows = 25 - len(column)
        first_not_recommended_numbers = euromillions.not_recommended_numbers.head(missing_rows)
        column = pd.concat([euromillions.recommended_numbers['numbers'], first_not_recommended_numbers.iloc[:, 0]]).reset_index(drop=True)
    else:
        result = np.isin(column,row).sum()
        success.append(result)
        failure.append(5 - result)

    # recommended_numbers = euromillions.recommended_numbers['numbers'].sample(6).tolist()
    # not_recommended_numbers = euromillions.not_recommended_numbers['numbers'].sample(4).tolist()
    # total_numbers = recommended_numbers + not_recommended_numbers
    # combinations = list(itertools.combinations(total_numbers, 5))

    # draw_results = []

    # for combination in combinations:
    #     counts = np.isin(row, combination).sum()
    #     draw_results.append(counts)

    # results.append(draw_results)
    # file_name = 'results.csv'
    # df_resultado = pd.DataFrame({'Combination': combinations, 'Count': draw_results})

    # Obtener el número máximo de conteo
    # max_count = max(draw_results)

    # if max_count == 0:
    #     print('FUCK!')
    # elif max_count == 1:
    #     print('Too bad')
    # elif max_count == 2:
    #     print('meh')
    # elif max_count == 3:
    #     print('now we are talking')
    # elif max_count == 4:
    #     print('holy shit!')
    # elif max_count == 5:
    #     print('HELL YEAH')

    sys.stdout.write(f"\ri = {draw}")
    sys.stdout.flush()
    
c = Counter(success)
x = Counter(failure)

# Print quantity of hits per draw, with recommended numbers, and random numbers aside
for i in range(0,6):
    hits = (success.count(i)/len(success))*100
    print(f"{i} hits: {c[i]}\n{round(hits,2)}%")

In [ ]:
euromillions.scrap

In [ ]:
random_succes = []
random_failure = []
for draw in draw_generator(size):
    db_resultados = db_slice.head(draw)
    row = euromillions.scrap.loc[draw,['nro1','nro2','nro3','nro4','nro5']]
    random_numbers = np.random.choice(range(1,51),size=25,replace=False)
    result = np.isin(row, random_numbers).sum()
    random_succes.append(result)
    random_failure.append(5-result)
    sys.stdout.write(f"\ri = {draw}")
    sys.stdout.flush()

y = Counter(random_succes)
z = Counter(random_failure)

# Print quantity of hits per draw, with recommended numbers, and random numbers aside
for e in range(0,6):
    nohits = (random_succes.count(e)/len(random_succes))*100
    print(f"{e} random hits: {y[e]}\n{round(nohits,2)}%")